# Sim-launcher
This script show how to launch sims using Python

### 1. Package imports

In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from pathlib import Path  # Python 3.6+ only
import os
import psycopg2
from psycopg2.extras import execute_values

### 2. Environment Variables

In [13]:
# Load the environment variables
env_path = Path('..') / '.env'
load_dotenv(dotenv_path=env_path)
# Print this to see if the env variables are read now
os.getenv("COMPOSE_PROJECT_NAME")

'evidss'

### 3. Database connection (writer)

In [4]:
# Generic function to test the connection to the database
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(    
            host=os.getenv("MAIN_HOST"),
            database=os.getenv("MAIN_DB"),
            user=os.getenv("DBWRITE_USER"),
            password=os.getenv("DBWRITE_PWD"), 
            port = os.getenv("MAIN_PORT")
        )
		
        # create a cursor
        cur = conn.cursor()
        
	# execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [5]:
# Make the test database connection
connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 12.3 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-11), 64-bit',)
Database connection closed.


In [28]:
conn = psycopg2.connect(    
    host=os.getenv("MAIN_HOST"),
    database=os.getenv("MAIN_DB"),
    user=os.getenv("DBWRITE_USER"),
    password=os.getenv("DBWRITE_PWD"), 
    port = os.getenv("MAIN_PORT")
)

# create a cursor
cur = conn.cursor()

### 4. Database queries

#### 4.1 Base-case analysis

In [29]:
sql_analysis = 'INSERT INTO analysis_record (user_id, status, include_tesla) VALUES (%s, %s, %s);'
analysis_data = (os.getenv("AUTH0_USERID"), 'inserted', 'FALSE')

In [30]:
sql_user = 'INSERT INTO user_details (user_id, user_name, email_id) VALUES (%s, %s, %s) ON CONFLICT (user_id) DO UPDATE SET last_submit_date = NOW();'
user_data = (os.getenv("AUTH0_USERID"), os.getenv("AUTH0_USERNAME"), os.getenv("AUTH0_USERNAME"))

In [31]:
sql_params = 'INSERT INTO analysis_params (param_id, param_value) VALUES %s'
params_data = [(1, '123' ),(2, '70' ),(14, '10' ),(3, '80' ),(4, '100' ),(9, '40' ),(10, '50' ),(11, '25' ),(12, '23' ),(13, '20' ), (15, '1' ), ( 16, '10' ), (17, '80' ), (18, '0' ), (19, '60' ), (20, '20' ), (21, '200' )]

In [33]:
# All these should be executed together as a transaction
cur.execute(sql_analysis,  analysis_data)
cur.execute(sql_user,  user_data)
execute_values(cur, sql_params, params_data)
conn.commit()

#### 4.2 Adding new chargers